In [1]:
from config import *
from data_processer import CSCDataset
from torch.utils.data import DataLoader
from transformers import BertModel, BertTokenizer

/root/anaconda3/envs/simple_csc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import (
    AdamW,
    BertForSequenceClassification,
    BertTokenizer,
    get_scheduler,
)

In [3]:
def cal_err(raw_sentence, pred_sentence, corr_sentence):
    matrices = ["over_corr", "total_err", "true_corr"]
    char_level = {key: 0 for key in matrices}
    sent_level = {key: 0 for key in matrices}

    f1 = f2 = 0
    for i, c in enumerate(raw_sentence):
        pc, cc = pred_sentence[i], corr_sentence[i]

        if cc != c:
            char_level["total_err"] += 1
            char_level["true_corr"] += pc == cc
            f1 = 1
        elif pc != cc:
            char_level["over_corr"] += 1
            f2 = 1

    if f1:
        sent_level["true_corr"] += all(pred_sentence == corr_sentence)
        sent_level["total_err"] += f1
    sent_level["over_corr"] += f2

    return char_level, sent_level

In [4]:
def test(model, tokenizer, test_data_loader):
    model.eval()
    total_loss = 0
    matrices = ["over_corr", "total_err", "true_corr"]
    test_char_level = {key: 0 for key in matrices}
    test_sent_level = {key: 0 for key in matrices}

    with torch.no_grad():
        for batch in test_data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device).type(torch.float)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, src_mask=attention_mask)
            logits = outputs.permute(0, 2, 1)

            loss = cross_entropy(logits, labels, ignore_index=tokenizer.pad_token_id)
            total_loss += loss.item()

            t = torch.argmax(outputs, dim=-1)
            nt = t * attention_mask
            pred = tokenizer.batch_decode(nt, skip_special_tokens=True)

            for i in range(len(t)):
                char_level, sent_level = cal_err(input_ids[i], nt[i], labels[i])
                test_char_level = {
                    key: test_char_level[key] + v for key, v in char_level.items()
                }
                test_sent_level = {
                    key: test_sent_level[key] + v for key, v in sent_level.items()
                }
        print(total_loss / len(test_data_loader), test_char_level, test_sent_level)

In [5]:
from config import *
from models import Seq2SeqModel
from torch.nn.functional import cross_entropy
from tqdm import tqdm

# temp_data
nhead = 4  # 多头注意力机制的头数
num_decoder_layers = 2
dim_feedforward = 3072
max_seq_len = 128
dropout = 0.1

if __name__ == "__main__":
    tokenizer = BertTokenizer.from_pretrained(checkpoint)
    bert = BertModel.from_pretrained(checkpoint)

    train_dataset = CSCDataset([SIGHAN_train_dir_err, SIGHAN_train_dir_corr], tokenizer)
    train_data_loader = DataLoader(
        train_dataset, num_workers=0, shuffle=True, batch_size=16
    )

    test_dataset = CSCDataset(
        [SIGHAN_train_dir_err14, SIGHAN_train_dir_corr14], tokenizer
    )
    test_data_loader = DataLoader(
        test_dataset, num_workers=0, shuffle=True, batch_size=32
    )

    model = Seq2SeqModel(
        bert, nhead, num_decoder_layers, dim_feedforward, max_seq_len, dropout
    )

    optimizer = AdamW(model.parameters(), lr=learning_rate)

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)

    epochs = 10
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(
            enumerate(train_data_loader),
            desc=f"Epoch:{epoch+1}/{epochs}",
            total=len(train_data_loader),
        )

        for i, batch in progress_bar:
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device).type(torch.float)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, src_mask=attention_mask)
            logits = outputs.permute(0, 2, 1)  # (batch_size, vocab_size, seq_len)

            # 反向传播在这，故labels不需要传入模型
            loss = cross_entropy(logits, labels, ignore_index=tokenizer.pad_token_id)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            progress_bar.set_postfix({"loss": "{:.3f}".format(loss.item())})

        t = torch.argmax(outputs, dim=-1)
        nt = t * attention_mask
        pred = tokenizer.batch_decode(nt, skip_special_tokens=True)
        # print(pred)
        # print(f"origin{tokenizer.batch_decode(labels, skip_special_tokens=True)}")

        for i, v in enumerate(nt):
            r, l = input_ids[i], labels[i]
            print(tokenizer.decode(r, skip_special_tokens=True))
            print(tokenizer.decode(v, skip_special_tokens=True))
            print(tokenizer.decode(l, skip_special_tokens=True))
            print(cal_err(r, v, l))

        print(f"Epoch {epoch+1} Loss: {total_loss / len(train_data_loader)}")
        test(model, tokenizer, test_data_loader)

preprocessing sighan dataset: 2339it [00:00, 921085.07it/s]
preprocessing sighan dataset: 100%|██████████| 2339/2339 [00:00<00:00, 1635079.51it/s]


共2339句，共73264字，最长的句子有171字


preprocessing sighan dataset: 3437it [00:00, 912509.36it/s]
preprocessing sighan dataset: 100%|██████████| 3437/3437 [00:00<00:00, 1333193.64it/s]


共3437句，共170330字，最长的句子有258字


/root/anaconda3/envs/simple_csc/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch:1/10: 100%|██████████| 147/147 [00:11<00:00, 13.22it/s, loss=2.377]
/root/anaconda3/envs/simple_csc/lib/python3.10/site-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(


...

想 說 白 天 大 家 都 上 班 ， 多 少 能 必 開 這 些 困 擾 但 晚 上 回 來 ， 原 本 可 以 好 好 休 息 、 放 鬆 自 己 ， 小 孩 也 能 好 好 做 功 課 ， 但 您 工 廠 又 進 行 加 班 。
想 說 白 天 大 家 都 上 班 ， 多 少 能 必 開 這 些 困 擾 但 晚 上 回 來 ， 原 本 可 以 好 好 休 息 、 放 鬆 自 己 ， 小 孩 也 能 好 好 做 功 課 ， 但 您 工 廠 又 進 行 加 班 。
想 說 白 天 大 家 都 上 班 ， 多 少 能 避 開 這 些 困 擾 但 晚 上 回 來 ， 原 本 可 以 好 好 休 息 、 放 鬆 自 己 ， 小 孩 也 能 好 好 做 功 課 ， 但 您 工 廠 又 進 行 加 班 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
因 為 時 間 的 關 係 ， 都 極 少 和 家 人 相 處 ， 就 趁 著 這 個 機 會 順 道 和 家 人 好 好 的 去 玩 玩 ， 放 鬆 一 下 心 情 ， 這 何 嘗 不 是 件 好 事 呢 。
因 為 時 間 的 關 係 ， 都 極 少 和 家 人 相 處 ， 就 趁 著 這 個 機 會 順 道 和 家 人 好 好 的 去 玩 玩 ， 放 鬆 一 下 心 情 ， 這 何 居 不 是 件 好 事 呢 。
因 為 時 間 的 關 係 ， 都 極 少 和 家 人 相 處 ， 就 趁 著 這 個 機 會 順 道 和 家 人 好 好 地 去 玩 玩 ， 放 鬆 一 下 心 情 ， 這 何 嘗 不 是 件 好 事 呢 。
({'over_corr': 1, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
老 師 說 ： 「 好 好 美 關 係 ， 明 天 你 得 來 ８ 點 好 不 。 」 。
老 師 說 ： 「 好 好 美 關 係 ， 明 天 你 得 

Epoch:9/10: 100%|██████████| 147/147 [00:09<00:00, 16.08it/s, loss=0.255]


「 家 張 限 制 青 少 年 上 網 時 數 」 的 話 題 ？ 我 非 常 支 持 ！
「 家 張 限 制 青 少 年 上 網 時 數 」 的 話 題 ？ 我 非 常 支 持 ！
「 家 長 限 制 青 少 年 上 網 時 數 」 的 話 題 ？ 我 非 常 支 持 ！
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
孩 子 先 在 也 要 上 高 中 了 吧 ， 你 還 可 以 輔 導 他 的 學 習 。
孩 子 先 在 也 要 上 高 中 了 吧 ， 你 還 可 以 輔 導 他 的 學 習 。
孩 子 現 在 也 要 上 高 中 了 吧 ， 你 還 可 以 輔 導 他 的 學 習 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
他 們 只 能 有 兩 個 選 擇 ： 接 受 降 新 或 自 動 離 職 。
他 們 只 能 有 兩 個 選 擇 ： 接 受 降 新 或 自 動 離 職 。
他 們 只 能 有 兩 個 選 擇 ： 接 受 降 薪 或 自 動 離 職 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
Epoch 9 Loss: 0.33716014686490403
nan {'over_corr': 5352, 'total_err': 5278, 'true_corr': tensor(378, device='cuda:0')} {'over_corr': 2016, 'total_err': 3436, 'true_corr': 80}


Epoch:10/10: 100%|██████████| 147/147 [00:09<00:00, 16.02it/s, loss=0.254]


這 樣 就 可 以 放 便 多 了 。 以 我 個 性 不 想 麻 煩 又 喜 歡 動 物 的 人 ， 這 重 設 備 是 滿 不 錯 的 。
這 樣 就 可 以 放 便 多 了 。 以 我 個 性 不 想 麻 煩 又 喜 歡 動 物 的 人 ， 這 重 設 備 是 滿 不 錯 的 。
這 樣 就 可 以 方 便 多 了 。 以 我 個 性 不 想 麻 煩 又 喜 歡 動 物 的 人 ， 這 種 設 備 是 滿 不 錯 的 。
({'over_corr': 0, 'total_err': 2, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
因 為 天 氣 很 好 ， 所 以 他 打 算 去 錄 行 。
因 為 天 氣 很 好 ， 所 以 他 打 算 去 錄 行 。
因 為 天 氣 很 好 ， 所 以 他 打 算 去 旅 行 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
不 上 課 的 時 候 ， 我 常 常 跟 朋 友 去 玩 玩 ， 看 一 看 在 台 北 好 看 的 地 方 ， 晚 上 我 就 看 書 跟 學 子 。
不 上 課 的 時 候 ， 我 常 常 跟 朋 友 去 玩 玩 ， 看 一 看 在 台 北 好 看 的 地 方 ， 晚 上 我 就 看 書 跟 學 子 。
不 上 課 的 時 候 ， 我 常 常 跟 朋 友 去 玩 玩 ， 看 一 看 在 台 北 好 看 的 地 方 ， 晚 上 我 就 看 書 跟 學 字 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
Epoch 10 Loss: 0.3092169188926009
nan {'over_corr': 4738, 'total_err': 5278,

In [6]:
for epoch in range(epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(
        enumerate(train_data_loader),
        desc=f"Epoch:{epoch+1}/{epochs}",
        total=len(train_data_loader),
    )

    for i, batch in progress_bar:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device).type(torch.float)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, src_mask=attention_mask)
        logits = outputs.permute(0, 2, 1)  # (batch_size, vocab_size, seq_len)

        # 反向传播在这，故labels不需要传入模型
        loss = cross_entropy(logits, labels, ignore_index=tokenizer.pad_token_id)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({"loss": "{:.3f}".format(loss.item())})

    t = torch.argmax(outputs, dim=-1)
    nt = t * attention_mask
    pred = tokenizer.batch_decode(nt, skip_special_tokens=True)
    # print(pred)
    # print(f"origin{tokenizer.batch_decode(labels, skip_special_tokens=True)}")

    for i, v in enumerate(nt):
        r, l = input_ids[i], labels[i]
        print(tokenizer.decode(r, skip_special_tokens=True))
        print(tokenizer.decode(v, skip_special_tokens=True))
        print(tokenizer.decode(l, skip_special_tokens=True))
        print(cal_err(r, v, l))

    print(f"Epoch {epoch+1} Loss: {total_loss / len(train_data_loader)}")
    test(model, tokenizer, test_data_loader)

Epoch:1/10: 100%|██████████| 147/147 [00:09<00:00, 16.11it/s, loss=0.330]


如 果 他 真 的 不 再 需 要 這 些 ， 他 起 碼 應 該 感 謝 你 ， 而 且 要 換 給 你 錢 。
如 果 他 真 的 不 再 需 要 這 些 ， 他 起 嗎 應 該 感 謝 你 ， 而 且 要 換 給 你 錢 。
如 果 他 真 的 不 再 需 要 這 些 ， 他 起 碼 應 該 感 謝 你 ， 而 且 要 還 給 你 錢 。
({'over_corr': 1, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
作 天 我 看 到 你 太 太 ， 她 告 訴 你 有 感 冒 。
作 天 我 看 到 你 太 太 ， 她 告 訴 你 有 感 冒 。
昨 天 我 看 到 你 太 太 ， 她 告 訴 你 有 感 冒 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
因 為 大 的 家 裡 學 校 很 遠 ， 所 以 他 要 早 一 點 起 來 。
因 為 大 的 家 裡 學 校 很 遠 ， 所 以 他 要 早 一 點 起 來 。
因 為 他 的 家 離 學 校 很 遠 ， 所 以 他 要 早 一 點 起 來 。
({'over_corr': 0, 'total_err': 2, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
Epoch 1 Loss: 0.2860999033361876
nan {'over_corr': 5122, 'total_err': 5278, 'true_corr': tensor(343, device='cuda:0')} {'over_corr': 1956, 'total_err': 3436, 'true_corr': 76}


Epoch:2/10: 100%|██████████| 147/147 [00:09<00:00, 16.11it/s, loss=0.304]


你 看 ， 你 的 情 況 不 是 那 嗎 誇 張 。
你 看 ， 你 的 情 況 不 是 那 嗎 誇 張 。
你 看 ， 你 的 情 況 不 是 那 麼 誇 張 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
第 四 個 象 片 兒 ： 兩 個 人 都 看 點 影 。
第 四 個 相 片 兒 ： 兩 個 人 都 看 點 影 。
第 四 個 相 片 兒 ： 兩 個 人 都 看 電 影 。
({'over_corr': 0, 'total_err': 2, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
接 者 因 為 民 眾 晚 上 十 點 之 後 希 望 能 夠 有 安 靜 既 舒 適 的 睡 眠 時 間 ， 本 人 也 得 知 貴 工 廠 是 裁 二 十 四 小 時 輪 班 運 作 方 式 ， 你 們 在 這 裡 開 工 廠 已 經 有 很 長 的 一 段 時 間 。
接 者 因 為 民 眾 晚 上 十 點 之 後 希 望 能 夠 有 安 靜 既 舒 適 的 睡 眠 時 間 ， 本 人 也 得 知 貴 工 廠 是 裁 二 十 四 小 時 輪 班 運 作 方 式 ， 你 們 在 這 裡 開 工 廠 已 經 有 很 長 的 一 段 時 間 。
接 著 因 為 民 眾 晚 上 十 點 之 後 希 望 能 夠 有 安 靜 及 舒 適 的 睡 眠 時 間 ， 本 人 也 得 知 貴 工 廠 是 採 二 十 四 小 時 輪 班 運 作 方 式 ， 你 們 在 這 裡 開 工 廠 已 經 有 很 長 的 一 段 時 間 。
({'over_corr': 0, 'total_err': 3, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
Epoch 2 Loss: 0.2648044910763397
n

Epoch:3/10: 100%|██████████| 147/147 [00:09<00:00, 16.07it/s, loss=0.256]


...

上 課 時 總 不 能 專 心 ， 因 為 會 覺 得 有 很 多 眼 睛 在 登 著 我 。
上 課 時 總 不 能 專 心 ， 因 為 會 覺 得 有 很 多 眼 睛 在 登 著 我 。
上 課 時 總 不 能 專 心 ， 因 為 會 覺 得 有 很 多 眼 睛 在 瞪 著 我 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
因 為 我 好 想 證 明 自 己 ， 所 以 拼 命 去 找 很 大 的 交 易 機 會 ， 結 果 檢 了 合 同 後 ， 才 發 現 自 己 被 騙 ， 總 價 值 和 痛 快 到 新 台 幣 五 億 元 。
因 為 我 好 想 證 明 自 己 ， 所 以 拼 命 去 找 很 大 的 交 易 機 會 ， 結 果 檢 了 合 同 後 ， 才 發 現 自 己 被 騙 ， 總 價 值 和 痛 快 到 新 台 幣 五 億 元 。
因 為 我 好 想 證 明 自 己 ， 所 以 拼 命 去 找 很 大 的 交 易 機 會 ， 結 果 簽 了 合 同 後 ， 才 發 現 自 己 被 騙 ， 總 價 值 合 同 快 到 新 台 幣 五 億 元 。
({'over_corr': 0, 'total_err': 3, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
我 從 小 一 直 看 著 你 經 營 公 司 的 樣 子 。 所 以 知 道 經 營 公 司 真 的 不 簡 單 ， 很 幸 苦 。
我 從 小 一 直 看 著 你 經 營 公 司 的 樣 子 。 所 以 知 道 經 營 公 司 真 的 不 簡 單 ， 很 幸 苦 。
我 從 小 一 直 看 著 你 經 營 公 司 的 樣 子 。 所 以 知 道 經 營 公 司 真 的 不 簡 單 ， 很 辛 苦 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, 

Epoch:9/10: 100%|██████████| 147/147 [00:09<00:00, 16.09it/s, loss=0.108]


這 樣 對 小 孩 子 的 安 全 好 ， 還 會 讓 父 母 請 放 心 。
這 樣 對 小 孩 子 的 安 全 好 ， 還 會 讓 父 母 請 放 心 。
這 樣 對 小 孩 子 的 安 全 好 ， 還 會 讓 父 母 親 放 心 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
因 為 又 大 聲 音 又 臭 味 ， 所 以 我 們 不 知 道 什 麼 告 訴 你 我 們 的 問 題 。
因 為 又 大 聲 音 又 臭 味 ， 所 以 我 們 不 知 道 什 麼 告 訴 你 我 們 的 問 題 。
因 為 又 大 聲 音 又 臭 味 ， 所 以 我 們 不 知 道 怎 麼 告 訴 你 我 們 的 問 題 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
老 闆 告 訴 王 大 華 幫 你 點 蔡 。
老 闆 告 訴 王 大 華 幫 你 點 菜 。
老 闆 告 訴 王 大 華 幫 你 點 菜 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
Epoch 9 Loss: 0.15367259951878567
nan {'over_corr': 19377, 'total_err': 5278, 'true_corr': tensor(529, device='cuda:0')} {'over_corr': 3259, 'total_err': 3436, 'true_corr': 19}


Epoch:10/10: 100%|██████████| 147/147 [00:09<00:00, 16.03it/s, loss=0.104]


因 為 德 國 的 天 氣 不 是 這 麼 熱 的 ， 也 不 太 施 。
因 為 德 國 的 天 氣 不 是 這 麼 熱 的 ， 也 不 太 施 。
因 為 德 國 的 天 氣 不 是 這 麼 熱 的 ， 也 不 太 濕 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
雖 然 對 我 來 說 太 大 而 不 是 很 方 便 ， 但 總 比 冷 死 好 。 就 這 樣 那 個 棉 被 默 默 的 用 到 現 在 了 。
雖 然 對 我 來 說 太 大 而 不 是 很 方 便 ， 但 總 比 冷 死 好 。 就 這 樣 那 個 棉 被 默 默 的 用 到 現 在 了 。
雖 然 對 我 來 說 太 大 而 不 是 很 方 便 ， 但 總 比 冷 死 好 。 就 這 樣 那 個 棉 被 默 默 地 用 到 現 在 了 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 0})
所 以 我 覺 得 除 非 接 受 安 寧 療 護 的 病 人 為 了 跟 寵 物 一 起 過 好 好 地 過 余 生 ， 也 可 以 利 用 這 種 寵 物 出 租 公 司 。
所 以 我 覺 得 除 非 接 受 安 寧 療 護 的 病 人 為 了 跟 寵 物 一 起 過 好 好 地 過 餘 生 ， 也 可 以 利 用 這 種 寵 物 出 租 公 司 。
所 以 我 覺 得 除 非 接 受 安 寧 療 護 的 病 人 為 了 跟 寵 物 一 起 過 好 好 地 過 餘 生 ， 也 可 以 利 用 這 種 寵 物 出 租 公 司 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 1, 'true_corr': 1})
Epoch 10 L

In [7]:
class Trainer:
    def __init__(self, model):
        self.model = model
        self.matrices = ["over_corr", "total_err", "true_corr"]

    def train(self, dataloader, epoch):
        self.iteration(dataloader, epoch)

    def test(self, dataloader):
        self.iteration(dataloader, train=False)

    def iteration(self, dataloader, epochs=1, train=True):
        mode = "train" if train else "dev"
        model.train() if train else model.eval()

        for epoch in range(epochs):
            # matrices
            total_loss = 0
            char_level = {key: 0 for key in self.matrices}
            sent_level = {key: 0 for key in self.matrices}

            progress_bar = tqdm(
                dataloader,
                desc=f"{mode} Epoch:{epoch+1}/{epochs}",
                total=len(dataloader),
            )
            for batch in progress_bar:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(input_ids, src_mask=attention_mask, tgt=labels)
                logits = outputs.permute(0, 2, 1)  # (batch_size, vocab_size, seq_len)

                loss = cross_entropy(
                    logits, labels, ignore_index=tokenizer.pad_token_id
                )

                total_loss += loss.item()
                progress_bar.set_postfix({"loss": "{:.3f}".format(loss.item())})

    def cal_err(raw_sentence, pred_sentence, corr_sentence):
        char_level = {key: 0 for key in self.matrices}
        sent_level = {key: 0 for key in self.matrices}

        for i, c in enumerate(raw_sentence):
            pc, cc = pred_sentence[i], corr_sentence[i]
            f1 = f2 = False

            if cc != c:
                char_level["total_err"] += 1
                char_level["true_corr"] += pc == cc
                f1 = True
            elif pc != cc:
                char_level["over_corr"] += 1
                f2 = True

        # true_corr 未计算
        sent_level["total_err"] += f1
        sent_level["over_corr"] += f2

        return char_level, sent_level